In [5]:
import pandas as pd
train=pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

## 1. 데이터 읽기, 전처리

In [6]:
# 1.<br/>과 같은 html 태그 삭제
# 2. 단어만을 남기고 문장부호나 숫자들을 삭제(regular expression)
# 3. nltk 패키지로 stopword 삭제

In [7]:
from bs4 import BeautifulSoup
import re
#!pip install nltk
import nltk

In [9]:
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\taeeu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [10]:
# 함수 생성
def review_to_words(raw_review):
  review_text=BeautifulSoup(raw_review, features="html.parser").get_text()
  letters_only=re.sub("[^a-zA-Z]"," ", review_text)
  words=letters_only.lower().split()
  stops=set(stopwords.words("english"))
  meaningful_words=[w for w in words if not w in stops]

  return(" ".join(meaningful_words))

In [11]:
# 실제로 train 데이터를 하나씩 review_to_words 함수를 통해서 clean_train_reviews
num_reviews=train["review"].size
clean_train_reviews=[]

for i in range(0, num_reviews):
  if ((i+1) % 1000==0):
    print("Review %d of %d\n" % (i + 1, num_reviews))
  clean_train_reviews.append(review_to_words(train["review"][i]))

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



## 2. Bag of Words
- Frequency based Embedding(단어로 이루어진 데이터를 수치화)

In [12]:
!pip install scikit-learn

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
# 단어중 빈도가 높은 5000개만 사용
vectorizer = CountVectorizer(analyzer = "word", tokenizer=None, preprocessor = None, stop_words=None, max_features=5000)

# fit_transform=vectorizer가 우리의 vocabulary를 익힘
train_data_features=vectorizer.fit_transform(clean_train_reviews)

# 다루기 쉽게 array로 변환
train_data_featrues=train_data_features.toarray()

## 3. Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
forest=RandomForestClassifier(n_estimators=100)
forest=forest.fit(train_data_features, train["sentiment"])

In [22]:
test=pd.read_csv("testData.tsv", header=0, delimiter='\t', quoting=3)
num_reviews=len(test["review"])
print(num_reviews)

25000


In [23]:
clean_test_reviews=[]

In [25]:
for i in range(0, num_reviews):
    if ((i+1) % 1000 == 0):
        print("Review %d of %d\n" % (i + 1, num_reviews))
    clean_review=review_to_words(test["review"][i])
    clean_test_reviews.append(clean_review)
test_data_features=vectorizer.transform(clean_test_reviews)
test_data_features=test_data_features.toarray()

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [26]:
result=forest.predict(test_data_features)
output=pd.DataFrame(data={"id": test["id"], "sentiment": result})

output.to_csv("Bag_of_Words_model.csv", index=False, quoting=3)